In [1]:
import sys
src_path =   '../src' # change as needed
sys.path.insert(0,src_path)

import numpy as np
import data_generator
import model_utils

INFO:absl:Using /tmp/tfhub_modules to cache modules.


In [2]:
#Start session
max_length = 128

train_data, val_data, test_data = data_generator.GetData(max_length, sample=1000)

100%|██████████| 3000/3000 [00:00<00:00, 3089.31it/s]


        tag  cat  occurences
0    B-MISC    0           4
1     I-LOC    1        2510
2    I-MISC    2        1137
3     I-ORG    3        1819
4     I-PER    4        8244
5         O    5       53099
6  [nerCLS]    6        3000
7  [nerPAD]    7      295291
8  [nerSEP]    8        3000
9    [nerX]    9       15896

                tag  cat  occurences
0  AFRICAN-AMERICAN    0        3405
1          EUROPEAN    1        1538
2         [raceCLS]    2        3000
3         [racePAD]    3      295291
4         [raceSEP]    4        3000
5           [raceX]    5       77766

           tag  cat  occurences
0       FEMALE    0        2838
1         MALE    1        2105
2  [genderCLS]    2        3000
3  [genderPAD]    3      295291
4  [genderSEP]    4        3000
5    [genderX]    5       77766



In [3]:
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

def getDebiasedModel(max_input_length, train_layers):
    
    in_id = tf.keras.layers.Input(shape=(max_input_length,), name="input_ids")
    in_mask = tf.keras.layers.Input(shape=(max_input_length,), name="input_masks")
    in_segment = tf.keras.layers.Input(shape=(max_input_length,), name="segment_ids")

    bert_inputs = [in_id, in_mask, in_segment]

    bert_sequence = model_utils.BertLayer(n_fine_tune_layers=train_layers)(bert_inputs)

    dense = tf.keras.layers.Dense(256, activation='relu', name='pred_dense')(bert_sequence)

    dense = tf.keras.layers.Dropout(rate=0.1)(dense)

    pred = tf.keras.layers.Dense(10, activation='softmax', name='ner')(dense)
    
    genderPred = tf.keras.layers.Dense(6, activation='softmax', name='gender')(pred)

    racePred = tf.keras.layers.Dense(6, activation='softmax', name='race')(pred)
    
    model = tf.keras.models.Model(inputs=bert_inputs, outputs={
        "ner": pred,
        "race": racePred,
        "gender": genderPred
    })
    
    model.summary()
    
    return model

In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

model = getDebiasedModel(max_length, 1)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert_layer (BertLayer)          (None, None, 768)    108931396   input_ids[0][0]                  
                                                                 input_masks[0][0]            

In [5]:
protect_loss_weight = 0.1
pred_learning_rate = 2**-16
protect_learning_rate = 2**-16
num_epochs = 8
batch_size = 32

In [6]:
def fit(train_data, epochs, batch_size, debias,
        protect_loss_weight = 0.1, 
        pred_learning_rate =  2**-16, 
        protect_learning_rate = 2**-16):

    num_train_samples = len(train_data["nerLabels"])

    ids_ph = tf.placeholder(tf.float32, shape=[batch_size, max_length])
    masks_ph = tf.placeholder(tf.float32, shape=[batch_size, max_length])
    sentenceIds_ph = tf.placeholder(tf.float32, shape=[batch_size, max_length])

    gender_ph = tf.placeholder(tf.float32, shape=[batch_size, max_length])
    ner_labels_ph = tf.placeholder(tf.float32, shape=[batch_size, max_length])

    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = 0.001
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 1000, 0.96, staircase=True)

    protect_vars = [var for var in tf.trainable_variables() if 'gender' in var.name]
    pred_vars = model.layers[3]._trainable_weights + [var for var in tf.trainable_variables() if any(x in var.name for x in ["pred_dense","ner"])]

    y_pred = model([ids_ph, masks_ph, sentenceIds_ph], training=True)

    ner_loss = model_utils.custom_loss(ner_labels_ph, y_pred["ner"])
    gender_loss = model_utils.custom_loss_protected(gender_ph, y_pred["gender"])

    protect_opt = tf.train.AdamOptimizer(protect_learning_rate)
    pred_opt = tf.train.AdamOptimizer(pred_learning_rate)

    protect_grads = {var: grad for (grad, var) in protect_opt.compute_gradients(gender_loss,var_list=pred_vars)}
    pred_grads = []

    tf_normalize = lambda x: x / (tf.norm(x) + np.finfo(np.float32).tiny)

    for (grad, var) in pred_opt.compute_gradients(ner_loss, var_list=pred_vars):
        unit_protect = tf_normalize(protect_grads[var])
        # the two lines below can be commented out to train without debiasing
        if debias:
            grad -= tf.reduce_sum(grad * unit_protect) * unit_protect
            grad -= tf.math.scalar_mul(protect_loss_weight, protect_grads[var])
        pred_grads.append((grad, var))

    pred_min = pred_opt.apply_gradients(pred_grads, global_step=global_step)
    protect_min = protect_opt.minimize(gender_loss, var_list=[protect_vars], global_step=global_step)

    model_utils.initialize_vars(sess)

    # Begin training
    for epoch in range(epochs):
            
        shuffled_ids = np.random.choice(num_train_samples, num_train_samples)

        for i in range(1, num_train_samples//32 + 1):
            
            batch_ids = shuffled_ids[batch_size*i: batch_size*(i+1)]

            batch_feed_dict = {ids_ph: train_data["inputs"][0][batch_ids], 
                               masks_ph: train_data["inputs"][1][batch_ids],
                               sentenceIds_ph: train_data["inputs"][2][batch_ids],
                               gender_ph: train_data["genderLabels"][batch_ids],
                               ner_labels_ph: train_data["nerLabels"][batch_ids]}

            _, _, pred_labels_loss_value, pred_protected_attributes_loss_vale = sess.run([
                pred_min,
                protect_min,
                ner_loss,
                gender_loss
            ], feed_dict=batch_feed_dict)
            

            if i % 10 == 0:
                print("epoch %d; iter: %d; batch classifier loss: %f; batch adversarial loss: %f" % (epoch, i, pred_labels_loss_value,
                                                                     pred_protected_attributes_loss_vale))


In [7]:
def print_status_bar(iteration, total, loss, metrics=None):
    metrics = " - ".join(["{}: {:.4f}".format(m.name, m.result())for m in [loss] + (metrics or [])])
    end = "" if iteration < total else "\n"
    print("\r{}/{} - ".format(iteration, total) + metrics, end=end)

In [9]:
fit(train_data, num_epochs, batch_size, True)

Epoch 0/8


AttributeError: 'numpy.float32' object has no attribute 'name'